
# 小地域の収入推定

```Data```フォルダには、小地域の収入データ(```income_district.csv```)と国勢調査の小地域集計データが格納されています。

国勢調査の小地域集計データ（人口構成、労働力構成、住宅形態など）から、その地域の収入を推定することが考えられます。推定のための機械学習・深層学習モデルを構築しなさい。


- データを観察・理解する上で、データの構造を説明しながら、適切なデータ整形を行いなさい
- データ構造や分析結果に対して、少なくとも二つの図で可視化を行いなさい
- モデルの精度を評価し、できるだけ精度が高いモデルを得るよう、適切な特徴量エンジニアリングやモデル選定の考えもまとめなさい


In [5]:
import pandas as pd

# ファイルの選択
directory = 'Data' # ディレクトリのパス

age_data_path = f'{directory}/h27_age_df.csv'
family_data_path = f'{directory}/h27_family_df.csv'
gender_data_path = f'{directory}/h27_gender_df2.csv'
housing_data_path = f'{directory}/h27_house_df.csv'
house_info_data_path = f'{directory}/h27_house_info_df.csv'
industry_data_path = f'{directory}/h27_indusry_df.csv'
occupation_data_path = f'{directory}/h27_job_df.csv'
labor_data_path = f'{directory}/h27_labor_df.csv'
marriage_data_path = f'{directory}/h27_marriage_df.csv'
employment_status_data_path = f'{directory}/h27_work_status_df.csv'
income_data_path = f'{directory}/income_district.csv'

age_data = pd.read_csv(age_data_path) #年齢層別人口、性別別人口などに関するデータ
family_data = pd.read_csv(family_data_path) #家族に関するデータ 
gender_data = pd.read_csv(gender_data_path) #総人口、男女別人口などに関するデータ
housing_data = pd.read_csv(housing_data_path) #住宅関連データ
house_info_data = pd.read_csv(house_info_data_path) #住居の階数別データ
industry_data = pd.read_csv(industry_data_path) #職業別男女人口データ
occupation_data = pd.read_csv(occupation_data_path) #職業別男女人口データ（職業分類が異なる）
labor_data = pd.read_csv(labor_data_path) #労働状況データ
marriage_data = pd.read_csv(marriage_data_path) #結婚状況データ
employment_status_data = pd.read_csv(employment_status_data_path) #就業形態データ
income_data = pd.read_csv(income_data_path) #地域コードと平均収入

In [6]:
# データフレームを結合する
# age_data,family_data
merged_data = pd.merge(age_data, family_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')

# 全てのデータ
merged_data = pd.merge(merged_data, gender_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, housing_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, house_info_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, industry_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, occupation_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, labor_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, marriage_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, employment_status_data, on=['district_id', 'district2_id', 'level_identifier', 'state_name', 'city_name', 'district_name', 'district2_name'], how='left')
merged_data = pd.merge(merged_data, income_data, left_on=['district_id'], right_on=['area_code'], how='left')

# 結合結果の確認
#merged_data.head()

# データをCSVファイルとして保存
#merged_data.to_csv('your_data_file.csv', index=False)

In [11]:
# 数値型の列だけを対象にする
numeric_cols = merged_data.select_dtypes(include=['number']).columns
for column in numeric_cols:
    # 無効なデータを NaN に変換
    merged_data[column] = pd.to_numeric(merged_data[column], errors='coerce')

# 欠損値の処理
merged_data[numeric_cols] = merged_data[numeric_cols].fillna(merged_data[numeric_cols].median())


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer

# 数値型の列だけを対象にする
numeric_cols = merged_data.select_dtypes(include=['number']).columns

# k-NN補完
knn_imputer = KNNImputer(n_neighbors=5)  # kの値（近傍数）を指定
merged_data[numeric_cols] = knn_imputer.fit_transform(merged_data[numeric_cols])

# ワンホットエンコーディング
merged_data_encoded = pd.get_dummies(merged_data, columns=['state_name', 'city_name', 'district_name', 'district2_name'])

# 特徴量とターゲットの分離
X = merged_data_encoded.drop(['income_mean'], axis=1)
y = merged_data_encoded['income_mean']

# データの標準化
scaler = StandardScaler()
X_standarized = scaler.fit_transform(X)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X_standarized, y, test_size=0.3, random_state=42)

# モデルの訓練
model = LinearRegression()
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# モデルの評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# 予測結果のプロット
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Income')
plt.ylabel('Predicted Income')
plt.title('Actual vs Predicted Income')
plt.show()


: 